In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from census import Census
import requests
import json
from uszipcode import SearchEngine
import gmaps

# Import census key and set to year 2016
from config_census import census_key
c = Census(census_key, year=2016)

# Import weather API key
from config_maps import api_key

#Import gmaps API key
from config_g import gkey

In [2]:
county_locations = pd.read_csv("Census_by_county.csv")
county_locations.head()

,County,Population,Poverty Count,No Degree,Degree,Lat,Lng
0,Abbeville County,20747.0,4669.0,5102.0,3220.0,34.23,-82.47
1,Ada County,429367.0,51360.0,64301.0,131300.0,43.42,-116.13
2,Adair County,72700.0,17031.0,17784.0,11932.0,40.20,-92.60
3,Adams County,768317.0,100228.0,172132.0,149855.0,39.87,-104.38
4,Addison County,35684.0,2861.0,7780.0,10793.0,44.05,-73.12


In [3]:
# Create Poverty Rate (Poverty Count / Population)
county_locations["Poverty Rate"] = 100 * county_locations["Poverty Count"].astype(int) / county_locations["Population"].astype(int)

# Add No Degree %
county_locations["No Degree %"] = county_locations["No Degree"] / (county_locations["No Degree"] + county_locations["Degree"])
county_locations["No Degree %"] = round(county_locations["No Degree %"]*100,2)

# Add Degree %
county_locations["Degree %"] = county_locations["Degree"] / (county_locations["No Degree"] + county_locations["Degree"])
county_locations["Degree %"] = round(county_locations["Degree %"]*100,2)

In [4]:
# Remove and reorder columns
county_locations = county_locations.reindex(columns=["County", "Population", "Poverty Rate", "No Degree", "Degree",
                                                      "No Degree %", "Degree %", "Lat", "Lng"])

county_locations.head()

,County,Population,Poverty Rate,No Degree,Degree,No Degree %,Degree %,Lat,Lng
0,Abbeville County,20747.0,22.504458,5102.0,3220.0,61.31,38.69,34.23,-82.47
1,Ada County,429367.0,11.961795,64301.0,131300.0,32.87,67.13,43.42,-116.13
2,Adair County,72700.0,23.426410,17784.0,11932.0,59.85,40.15,40.20,-92.60
3,Adams County,768317.0,13.045136,172132.0,149855.0,53.46,46.54,39.87,-104.38
4,Addison County,35684.0,8.017599,7780.0,10793.0,41.89,58.11,44.05,-73.12


In [5]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

# Store 'Lat' and 'Lng' into  locations 
locations = county_locations[["Lat", "Lng"]].astype(float)

# Store Poverty Rate
degrees = county_locations["Degree %"].astype(float)

In [70]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=degrees)

heat_layer.dissipating=False
heat_layer.max_intensity = 100
heat_layer.point_radius = 1
#heat_layer.opacity = 0.0
fig.add_layer(heat_layer)
heat_layer.gradient = [(200, 200, 200, 0.6), (0,255,255),(0,0,255)]

fig

Figure(layout=FigureLayout(height='420px'))

In [50]:
more_degree_counties = county_locations.loc[county_locations["Degree %"] > 50]
less_degree_counties = county_locations.loc[county_locations["Degree %"] < 50]
more_degree_plots = more_degree_counties[["Lat", "Lng"]]
less_degree_plots = less_degree_counties[["Lat", "Lng"]]

In [62]:
more_degree_layer = gmaps.symbol_layer(more_degree_plots, fill_color = 'rgb(0,128,0)',
                               stroke_color='rgb(0,128,0)', scale=2)

fig = gmaps.figure()
fig.add_layer(more_degree_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [64]:
less_degree_layer = gmaps.symbol_layer(less_degree_plots, fill_color = 'rgb(128,0,0)',
                               stroke_color='rgb(128,0,0)', scale=2)

fig = gmaps.figure()
fig.add_layer(less_degree_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [69]:
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(more_degree_layer)
fig.add_layer(less_degree_layer)
fig

Figure(layout=FigureLayout(height='420px'))